# Panda experiments - multiple rows to columns

## Try using cumcount and unstack

In [1]:
import pandas as pd

f = r"pivot_test.csv"
df = pd.read_csv(f)
df.head()

,ID,CLASS,PERCENT
0,A,Z,12
1,A,Z,20
2,A,Y,8
3,A,X,3
4,A,X,4


In [2]:
df_grp = df.groupby(['ID', 'CLASS']).sum()
df_grp

PERCENT
ID CLASS         
A  X            7
   Y            8
   Z           32
B  X           12
   Y            7
   Z           29

In [3]:
df_grp["LC_FLD"] = "LandCover" + (df_grp.groupby(level=0).cumcount()).astype(str)
df_grp

PERCENT      LC_FLD
ID CLASS                     
A  X            7  LandCover0
   Y            8  LandCover1
   Z           32  LandCover2
B  X           12  LandCover0
   Y            7  LandCover1
   Z           29  LandCover2

In [4]:
df_grp = df_grp.reset_index().set_index(['ID', 'LC_FLD'])
df_grp


CLASS  PERCENT
ID LC_FLD                   
A  LandCover0     X        7
   LandCover1     Y        8
   LandCover2     Z       32
B  LandCover0     X       12
   LandCover1     Y        7
   LandCover2     Z       29

In [5]:
unstack_df = df_grp.unstack()
unstack_df

CLASS                          PERCENT                      
LC_FLD LandCover0 LandCover1 LandCover2 LandCover0 LandCover1 LandCover2
ID                                                                      
A               X          Y          Z          7          8         32
B               X          Y          Z         12          7         29

In [6]:
unstack_df.columns = unstack_df.columns.map('_'.join)
unstack_df

,CLASS_LandCover0,CLASS_LandCover1,CLASS_LandCover2,PERCENT_LandCover0,PERCENT_LandCover1,PERCENT_LandCover2
ID,,,,,,
A,X,Y,Z,7,8,32
B,X,Y,Z,12,7,29


## Try using Lists

In [7]:
import pandas as pd

f = r"D:\projects\landcover\investigations\pandas_pivot\pivot_test.csv"
df = pd.read_csv(f)
df.head()

,ID,CLASS,PERCENT
0,A,Z,12
1,A,Z,20
2,A,Y,8
3,A,X,3
4,A,X,4


In [8]:
df_grp = df.groupby(['ID', 'CLASS']).sum()
df_grp = df_grp.reset_index()
df_grp

,ID,CLASS,PERCENT
0,A,X,7
1,A,Y,8
2,A,Z,32
3,B,X,12
4,B,Y,7
5,B,Z,29


In [9]:
list_df = df_grp.groupby(['ID'])[['CLASS','PERCENT']].agg(lambda x: list(x))
list_df = list_df.reset_index()
list_df

,ID,CLASS,PERCENT
0,A,"[X, Y, Z]","[7, 8, 32]"
1,B,"[X, Y, Z]","[12, 7, 29]"


In [10]:
cl_df = pd.DataFrame(list_df['CLASS'].to_list()).add_prefix('LandCover')
cl_df

,LandCover0,LandCover1,LandCover2
0,X,Y,Z
1,X,Y,Z


In [11]:
per_df = pd.DataFrame(list_df['PERCENT'].to_list()).add_prefix('LandCoverPercent')
per_df

,LandCoverPercent0,LandCoverPercent1,LandCoverPercent2
0,7,8,32
1,12,7,29


In [12]:
final_df = df = pd.concat([list_df, cl_df, per_df], axis=1)
final_df

,ID,CLASS,PERCENT,LandCover0,LandCover1,LandCover2,LandCoverPercent0,LandCoverPercent1,LandCoverPercent2
0,A,"[X, Y, Z]","[7, 8, 32]",X,Y,Z,7,8,32
1,B,"[X, Y, Z]","[12, 7, 29]",X,Y,Z,12,7,29
